# Requirements

### Models:
You need to download the sent2vec (SentEmbedding) model of hazm from this [link](https://mega.nz/file/WzR0QChY#J1nG-HGq0UJP69VMY8I1YGl_MfEAFCo5iizpjofA4OY) and extract it in a directory named `sent2vec`.

### Libraries:

In [13]:
!pip install -r requirements.txt
!pip install perke
!python -m perke download

Downloading...
From: https://drive.google.com/uc?id=1Q3JK4NVUC2t5QT63aDiVrCRBV225E_B3
To: C:\Users\LENOVO\anaconda3\envs\Desktop\lib\site-packages\perke\resources\pos_tagger.model

  0%|          | 0.00/19.2M [00:00<?, ?B/s]
  3%|2         | 524k/19.2M [00:01<00:42, 436kB/s]
  5%|5         | 1.05M/19.2M [00:01<00:27, 656kB/s]
  8%|8         | 1.57M/19.2M [00:01<00:18, 953kB/s]
 11%|#         | 2.10M/19.2M [00:02<00:12, 1.38MB/s]
 14%|#3        | 2.62M/19.2M [00:02<00:09, 1.78MB/s]
 16%|#6        | 3.15M/19.2M [00:02<00:08, 1.99MB/s]
 19%|#9        | 3.67M/19.2M [00:02<00:08, 1.82MB/s]
 22%|##1       | 4.19M/19.2M [00:02<00:07, 2.03MB/s]
 25%|##4       | 4.72M/19.2M [00:03<00:07, 1.88MB/s]
 27%|##7       | 5.24M/19.2M [00:03<00:07, 1.81MB/s]
 30%|##9       | 5.77M/19.2M [00:03<00:06, 2.21MB/s]
 33%|###2      | 6.29M/19.2M [00:03<00:05, 2.27MB/s]
 35%|###5      | 6.82M/19.2M [00:04<00:05, 2.47MB/s]
 38%|###8      | 7.34M/19.2M [00:04<00:05, 2.22MB/s]
 44%|####3     | 8.39M/19.2M [00:04<0

# Data : Persian Technological News Articles

I have determined the data using some crawling. I chose a website (with its linked and child websites) about technology articles named **zoomit**. I crawled the articles of zoomit and its subbrands (with their own webpages), to have some Persian news mixed with some English phrases and entities about technology. Let me explain a little about that:


## [zoomit](https://zoomit.ir)

Zoom Media Group consisted of three large media brands: Zoomit (Technology, Car and Science), Zoomg (Entertainment including Gaming, Cinema, and TV), and Pedal (Cars).

Zoomit is the most popular technology magazine in Iran with more than 40 million monthly Pageviews. Zoomit covers the latest news and in-depth reviews of Mobiles, laptops, gadgets, other Consumer Electronic Devices in the form of videos, pictures, and text. Zoomit expanded its business by adding a price aggregator service named Zoomit’s Product in 2019. Product Section became a complete reference for digital products with links to the best Online Shops in the market. Zoomit is dominant in Google Search Results for related keywords with more than 100 million monthly page impressions.

Zoomg is another subbrand in the Zoom Media Group launched in 2014. Zoomg publishes content in Entertainment includes News and Reviews of video games, movies, and TV series. Rich and unique content caused Zoomg to experience rapid growth and becoming the most prominent player in its field in the market.


## crawling

Now, I want to explain how I've crawled the website and scraped them: (<font color='yellow'>The crawling script and its corresponding files like the extracted data which will be used for further tasks, are located in the `crawling` directory</font>)

As the webpages I wanted to scrape are Single Page Applications (SAPs), I had to use a webdriver for the purpose. So, I used Selenium and a Chromedriver compatible with my Chrome browser version.

The script as explained, is designed to scrape webpages of articles from a 'zoomit', which may include some other subbrands like 'zoomg' or 'pedal' webpages. It utilizes Selenium along with ChromeDriver for automation. 

Upon initialization, the script sets up necessary configurations such as the path to the ChromeDriver executable, the starting URL, and the file path for storing scraped data.

To start the scraping process, it launches a Selenium service and loads the initial URL. It then employs dynamic interaction by scrolling down and clicking a "load more" button to reveal additional articles. 

Once the articles are loaded, the script extracts their URLs. It then iterates through these URLs, navigating to each article's page. 

During this process, it distinguishes between articles belonging to the main 'zoomit' brand and those associated with subbrands. For 'zoomit' articles, it extracts relevant data such as title, content, tags, and source page URL using specific XPaths. 

In cases where articles belong to subbrands, such as 'zoomg' 'pedal', or 'zoomit2', the script follows a similar procedure to extract the necessary information.

Throughout the scraping process, error handling mechanisms are implemented to manage any exceptions that may occur.

Finally, the extracted data is stored in a JSON file named `data.json` for further analysis or processing. It has 4 fields of 'title', 'content', 'tags', and 'source' for each article, in which we just work with the text of article or 'content' field of that. Here is the structure of each item of the data:

| Attribute |                       Description                       | Type  |
|:---------:|:-------------------------------------------------------:|:-----:|
|  `title`  |                      article title                      |*string*|
| `content` |           content and description of article            | *string*|
|  `tags`   | list of associated tags on the website for each article | *list(string)*|
| `source`  |                       article url                       | *string*|



# Preprocess: Normalization

The articles in the zoomit, mostly don't need specific normalizations like fixing half spaces or removing unwanted characters (َ ، ِ ، ُ ، ...); and just need normalizations like fixing the position of some punctuations, converting English numbers to persian and... . But, for the normalization to be more challenging, <font color='red'>when scraping the webpages to gather the data, I converted all the half spaces (`\u200c` characters) to spaces and then, saved them.</font> So, the `data.json` file needs many normalizations which are challenging, because for example there is no half spaces in that and all of them should be fixed.

The normalization part, consists of 4 classes in 4 .py files which all together do the normalization of text: (<font color='yellow'>Some of the classes use some data files located in the `preprocessing/HalfSpace_Data` directory. They are `all_verbs.txt` which contains a huge set of all Persian verbs, `prefix.txt` and `suffix.csv` which contain Persian prefixes and suffixes with their specific attributes and constraints explained further</font> )
1. **normalizer:**  Each part of the project has been developed as a class, which we will discuss below. As the first class, we should start with the Normalizer class. The duty of this class is to correct general issues and somewhat preprocess the input text. This class includes two methods named "characterRefine" and "punctuationRefine" which are responsible for the following tasks:

- Cleaning up extra spaces in the text (converting multiple spaces, line breaks, and half spaces into one).
- Adhering to spacing rules regarding punctuation marks.
- Changing numbers and English symbols to Persian script.
- Changing quotation marks to Persian ones.
- Removing diacritics like (َ ، ِ ، ُ ، ...).
- Cleaning up emphasized letters and beautification cases (converting "ســــلام" to "سلام").
- Changing Arabic "ی" and "ک" to Persian "ی" and "ک".
  
  More details can be seen in the `normalizer.py` file as the normalizations which it does, are explained in the comments.

2. **verb:** The class "verb" is designed for processing verbs written in the text and fixing half spaces in them like 'می کرده اید' to 'می‌کرده‌اید'. This class only has one function. The duty of this function is to read all verbs collected and search for them in the given text file named `all_verbs.txt`. The body of this function includes all necessary regular expressions for extracting different forms of verbs.

    More details can be seen in the `verb.py` file as the normalizations which it does, are explained in the comments.


3. **regexer:** The duty of this class is to load rules from rule files, convert them into regular expressions, and compile them. Each function within this class is developed for a category of rules. For example, the "suffixPatternGenerator" function is responsible for performing the mentioned operations on suffix words. After reading the rules from the file, this function processes them into an array and generates regular expressions based on the type of rules. Then it sends the rules to the "compilePatterns" function to compile them, and the output is compiled regular expressions. At the end, we have fixed the spacing in all the words containing suffixes and prefixes.  

There are some notes that you should consider about this class:

- The suffix.txt and prefix.txt files are implemented such that each item has a type: It may be an affix denoted by 'a' which means it needs no character within its root, or may need a half space to be put between it and its root. this type is denoed by 'h'.
- Also, each item has its own constraints. For example, if a root has just 3 characters and doesn't start with some specific letters, it should have no space between it and its prefix. As another example, the suffixes of 'تر' and 'ترین' and their roots, should always have a half space between, but if the root is 'بیش' or 'به', they must be attached to their roots with no half space between. Other rules can be seen as commented in the `regexer.py` file.
- #### All the regular expressions, types of prefixes and suffixes, and specific constraints of them, are determined by so much time I have spent exploring many websites and consulting with a professor of Persian literature, Dr. Saeid Bahabadi.  

4. **spacing:** Finally, the "Spacing" class is responsible for examining all extra letters except for verbs. After the verbs are corrected, the text is sent to this class to examine other cases. The functions inside this class are categorized based on whether the words are prefixes, suffixes, or neither. The duty of these functions is to receive patterns from the Regexer class and search for them within the text. Finally, they replace the found patterns with the correct ones. 


#### At the end, `preprocessor.py` reads the unprocessed `data.json` file and processes each item of that to be completely normalized and saves the processed data into a file named `processed_data.json` to be used for keyphrase extraction.

<font color='red'>All the above files are implemented by myself except the `normalizer.py`, `verb.py` and `all_verbs.txt` which are taken from a [Github](https://github.com/PouyaKhn/NLP_HWs/tree/main/HW1/Solutions/Verb_Processing) repository with some little edits for my purpose.</font>


## One Sample of Unprocessed Data

In [2]:
import json

with open('crawling/data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    
print(data[3]['content'])

 اپل سرانجام چندی پیش هدست واقعیت ترکیبی ویژن پرو را که از مدت ها وعده داده بود، عرضه کرد. این شرکت دستگاه جدیدش را که قیمت اولیه ی آن ۳۵۰۰ دلار است، به عنوان «کامپیوتر فضایی» و جایگزینی برای لپ تاپ ها یا کامپیوترهای رومیزی استاندارد توصیف می کند. تبلیغات اپل افرادی را نشان می دهد که از هدست ویژن پرو برای ارسال ایمیل و سایر کارهای دو بعدی معمولی استفاده می کنند و این شرکت در بیانیه مطبوعاتی ژوئن ۲۰۲۳ اعلام کرد ویژن پرو برای استفاده تمام روز طراحی شده است. کاربران مشتاق اولیه گزارش می کنند که از آن ده ها ساعت به طور مداوم استفاده می کنند. به  نقل از ساینتیفیک آمریکن، بسیاری از کارشناسان تردید دارند که هدست هایی همچون ویژن پرو بتوانند یا باید جایگزین مانیتورهای فیزیکی، صفحه کلید و موس شود. برخی نگران هستند که استفاده ی طولانی مد ت از چنین دستگاهی بتواند به بیماری حرکت، انواع جدیدی از انزوای اجتماعی یا سایر پیامدهای ناخواسته منجر شود. جرمی بيلنسون، مدیر مؤسس آزمایشگاه تعامل انسانی مجازی دانشگاه استنفورد که روانشناسی واقعیت مجازی و واقعیت افزوده را مطالعه می کند، می گوید: «واقعیت مجازی و و

## One Sample of Processed Data (Normalized Data)

In [1]:
import json

with open('preprocessing/processed_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    
print(data[3]['content'])

 اپل سرانجام چندی پیش‌هدست واقعیت ترکیبی ویژن پرو را که از مدت‌ها وعده داده‌بود، عرضه کرد. این شرکت دستگاه جدیدش را که قیمت اولیه‌ی آن ۳۵۰۰ دلار است، به عنوان «کامپیوتر فضایی» و جایگزینی برای لپ تاپ‌ها یا کامپیوترهای رومیزی استاندارد توصیف می‌کند. تبلیغات اپل افرادی را نشان می‌دهد که از هدست ویژن پرو برای ارسال ایمیل و سایر کارهای دو بعدی معمولی استفاده می‌کنند و این شرکت در بیانیه مطبوعاتی ژوئن ۲۰۲۳ اعلام کرد ویژن پرو برای استفاده تمام روز طراحی شده‌است. کاربران مشتاق اولیه گزارش می‌کنند که از آن ده‌ها ساعت به طور مداوم استفاده می‌کنند. به نقل از ساینتیفیک آمریکن، بسیاری از کارشناسان تردید دارند که هدست‌هایی همچون ویژن پرو بتوانند یا باید جایگزین مانیتورهای فیزیکی، صفحه کلید و موس شود. برخی نگران هستند که استفاده‌ی طولانی مد ت از چنین دستگاهی بتواند به بیماری حرکت، انواع جدیدی از انزوای اجتماعی یا سایر پیامدهای ناخواسته منجر شود. جرمی بيلنسون، مدیر مؤسس آزمایشگاه تعامل انسانی مجازی دانشگاه استنفورد که روانشناسی واقعیت مجازی و واقعیت افزوده را مطالعه می‌کند، می‌گوید: «واقعیت مجازی و وا

As you can see, all the half spaces are fixed for verbs or suffixes or prefixes, and all the other normalizations which were mentioned are done correctly like fixing the space between '(' punctuation and its previous word. 

## Explore the Data

Now, we are going to explore the data and extract some information like frequency analysis inspiring from the NoteBook introduced in the homework pdf file. 

### Sentence Tokenization

In [4]:
from hazm import *

contents = [item['content'] for item in data]

contents_sentences = [sent_tokenize(x) for x in contents]
contents_sentences[0]

['سامسونگ در نوامبر ۲۰۲۳(آبان و آذر ۱۴۰۲) گوشی گلکسی A05 را در هند عرضه کرد.',
 'حالا غول کره\u200cای قیمت گوشی را در هند کاهش داده\u200cاست.',
 'قیمت جدید، هر دو پیکربندی رم و فضای ذخیره\u200cسازی را شامل می\u200cشود.',
 'سامسونگ گلکسی A05 در پیکربندی ۴ گیگابایت رم و ۶۴ گیگابایت فضای ذخیره\u200cسازی و ۶ گیگابایت رم با ۱۲۸ گیگابایت فضای ذخیره\u200cسازی عرضه شده\u200cاست.',
 'کانفیگ\u200cهای مورد بحث گوشی سامسونگ در زمان عرضه به ترتیب ۱۲۰ و ۱۵۷ دلار قیمت داشتند.',
 'اکنون، مدل پایه با قیمت ۱۰۴ دلار به فروش می\u200cرسد و قیمت گوشی در مدل ۱۲۸ گیگابایتی به ۱۳۳ دلار کاهش یافته\u200cاست.',
 'علاقه مندان در هند می\u200cتوانند سامسونگ گلکسی A05 را از وب سایت رسمی سامسونگ و وب سایت Croma خریداری کنند.',
 'این گوشی در رنگ\u200cهای سبز روشن، مشکی و نقره\u200cای عرضه می\u200cشود.',
 'گلکسی A05 دارای صفحه نمایش ۶٫۷ اینچی با وضوح +HD و ناچ Infinity-U است که یک دوربین سلفی ۸ مگاپیکسلی را در خود جای داده.',
 'در پشت گوشی، دوربین اصلی ۵۰ مگاپیکسلی با یک حسگر عمق سنج ۲ مگاپیکسلی قرار گرفته\u200cاست.',
 

### Tokenization

In [5]:
tokens = [[word_tokenize(sent) for sent in sents] for sents in contents_sentences]
tokens[0]

[['سامسونگ',
  'در',
  'نوامبر',
  '۲۰۲۳',
  '(',
  'آبان',
  'و',
  'آذر',
  '۱۴۰۲',
  ')',
  'گوشی',
  'گلکسی',
  'A',
  '05',
  'را',
  'در',
  'هند',
  'عرضه',
  'کرد',
  '.'],
 ['حالا',
  'غول',
  'کره\u200cای',
  'قیمت',
  'گوشی',
  'را',
  'در',
  'هند',
  'کاهش',
  'داده\u200cاست',
  '.'],
 ['قیمت',
  'جدید',
  '،',
  'هر',
  'دو',
  'پیکربندی',
  'رم',
  'و',
  'فضای',
  'ذخیره\u200cسازی',
  'را',
  'شامل',
  'می\u200cشود',
  '.'],
 ['سامسونگ',
  'گلکسی',
  'A',
  '05',
  'در',
  'پیکربندی',
  '۴',
  'گیگابایت',
  'رم',
  'و',
  '۶۴',
  'گیگابایت',
  'فضای',
  'ذخیره\u200cسازی',
  'و',
  '۶',
  'گیگابایت',
  'رم',
  'با',
  '۱۲۸',
  'گیگابایت',
  'فضای',
  'ذخیره\u200cسازی',
  'عرضه',
  'شده\u200cاست',
  '.'],
 ['کانفیگ\u200cهای',
  'مورد',
  'بحث',
  'گوشی',
  'سامسونگ',
  'در',
  'زمان',
  'عرضه',
  'به',
  'ترتیب',
  '۱۲۰',
  'و',
  '۱۵۷',
  'دلار',
  'قیمت',
  'داشتند',
  '.'],
 ['اکنون',
  '،',
  'مدل',
  'پایه',
  'با',
  'قیمت',
  '۱۰۴',
  'دلار',
  'به',
  'فروش',
  'م

### Frequency Analysis

In [6]:
from nltk import FreqDist
import itertools
import pandas as pd

all_tokens = list(itertools.chain(*itertools.chain(*tokens)))

tokens_df = {}

tokens_df['all_tokens'] = FreqDist(eval("all_tokens")).most_common(25)

freq_analysis = pd.DataFrame(tokens_df)   
freq_analysis.head(20)

,all_tokens
0,"(., 11635)"
1,"(و, 8917)"
2,"(به, 8574)"
3,"(،, 7744)"
4,"(در, 7550)"
5,"(از, 6506)"
6,"(را, 4873)"
7,"(که, 4864)"
8,"(با, 4597)"
9,"(این, 3635)"


In [7]:
import numpy as np

print ('%-16s' % 'Number of words', '%-16s' % len(all_tokens))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(all_tokens)))
avg=np.sum([len(word) for word in all_tokens])/len(all_tokens)
print ('%-16s' % 'Average word length', '%-16s' % avg)
print ('%-16s' % 'Longest word', '%-16s' % all_tokens[np.argmax([len(word) for word in all_tokens])])

Number of words  291002          
Number of unique words 16243           
Average word length 4.057164555570065
Longest word     RECEIVE_SENSITIVE_NOTIFICATIONS


# Task : Keyphrase Extraction

## <font color='red'>Wrong Approaches:</font>
In the way of determining the correct approach, I tested many other approaches which are brought as expected. I used some libraries and their different keyphrase extraction algorithms but the results were not quite satisfying. Here, is the results for the algorithms of perke library:
 
### TextRank Algorithm

In [5]:
from perke.unsupervised.graph_based import TextRank

# refine all
for item in data[:4]:
    text = item['content']
    #text = " ".join([w for w in text.split() if w not in stop_words])
    valid_pos_tags = {'NOUN', 'ADJ'}
    # 1. Create a TextRank extractor.
    extractor = TextRank(valid_pos_tags=valid_pos_tags)

# 2. Load the text.
    extractor.load_text(input=text)

# 3. Build the graph representation of the text and weight the
#    words. Keyphrase candidates are composed of the 33 percent
#    highest weighted words.
    extractor.weight_candidates(window_size=2, top_t_percent=0.33)

# 4. Get the 10 highest weighted candidates as keyphrases.
    keyphrases = extractor.get_n_best(n=10)
    
    for word, score in keyphrases:
        print(word)
    
    print('\n\n')

قیمت گوشی
گوشی گلکسی
گوشی سامسونگ
بهترین گوشی
به کاری سرگرم‌کننده
قیمت جدید
سامسونگ گلکسی
عرضه به
گوشی
سامسونگ دارای


کاربری Google Contacts
گوشی هوشمند رول شدنی
برنامه‌ی Google Contacts
شدن ویژگی‌های جدید
نزدیک شاهد تغییرات
تلویزیون‌های سری QNED
Google Contacts
شرایط تغییر
جدید شیائومی
کاربری نوار


شیائومی مدل جدیدی
گوشی سامسونگ مدل گلکسی
گوشی جدیدی
به‌روزرسانی جدید
جدید شیائومی
به‌روزرسانی مشکل دوربین
نمایش سری گلکسی
تلویزیون‌های سری QNED
مدت طولانی موسیقی
بازار کره‌ی جنوبی


هدست واقعیت مجازی استفاده
هدست‌های واقعیت ترکیبی استفاده
دستگاه واقعیت مجازی رانندگی
هدست‌های واقعیت ترکیبی هنگام رانندگی
واقعیت ترکیبی ویژن پرو را
هدست واقعیت مجازی
مدت واقعیت ترکیبی را
رفتاری واقعیت مجازی
تعامل انسانی مجازی دانشگاه
هدست واقعیت ترکیبی


### TopicRank Algorithm

In [7]:
from perke.unsupervised.graph_based import TopicRank

# refine all
for item in data[:4]:
    text = item['content']
    #text = " ".join([w for w in text.split() if w not in stop_words])

    # Define the set of valid part of speech tags to occur in the model.
    valid_pos_tags = {'NOUN', 'ADJ'}

    # 1. Create a TopicRank extractor.
    extractor = TopicRank(valid_pos_tags=valid_pos_tags)

# 2. Load the text.
    extractor.load_text(input=text, word_normalization_method=None)

# 3. Select the longest sequences of nouns and adjectives, that do
#    not contain punctuation marks or stopwords as candidates.
    extractor.select_candidates()

# 4. Build topics by grouping candidates with HAC (average linkage,
#    jaccard distance, threshold of 1/4 of shared normalized words).
#    Weight the topics using random walk, and select the first
#    occurring candidate from each topic.
    extractor.weight_candidates(
        threshold=0.75, metric='jaccard', linkage_method='average'
    )

# 4. Get the 10 highest weighted candidates as keyphrases.
    keyphrases = extractor.get_n_best(n=10)
    
    for word, score in keyphrases:
        print(word)
    
    print('\n\n')

گوشی
دلار قیمت
مگاپیکسلی
قیمت
مدل پایه
هند عرضه
دوربین سلفی
هند کاهش
فروش
گیگابایتی


منوی همبرگری حذف
ارسال پیامک نمایش
رابط کاربری Google Contacts
گوگل
برنامه مجهز
لگی کار
نشان
معرفی
موسوم
Phantom Ultimate


دوربین
ویژگی جدیدی
خانواده منتشر
سامسونگ
انتشار
رفع
دسترس کاربران اروپایی قرار
کاربرانی
تمایل
مراجعه


روانشناسی واقعیت مجازی
بعدی معمولی استفاده
واقعیت ترکیبی ویژن پرو
هدست ویژن پرو
هدست
دستگاه مجهز
جرمی بیلنسون
نشان
چشم
واقعیت افزوده


### Singlerank

In [8]:
from perke.unsupervised.graph_based import SingleRank

# refine all
for item in data[:4]:
    text = item['content']
    #text = " ".join([w for w in text.split() if w not in stop_words])

    # Define the set of valid part of speech tags to occur in the model.
    valid_pos_tags = {'NOUN', 'NOUN,EZ', 'ADJ', 'ADJ,EZ'}

    # 1. Create a TopicRank extractor.
    extractor = SingleRank(valid_pos_tags=valid_pos_tags)

# 2. Load the text.
    extractor.load_text(input=text, word_normalization_method='lemmatization')

# 3. Select the longest sequences of nouns and adjectives as
#    candidates.
    extractor.select_candidates()

# 4. Weight the candidates using the sum of their words weights that
#    are computed using random walk. In the graph, nodes are certain
#    parts of speech (nouns and adjectives) that are connected if
#    they co-occur in a window of 10 words.
    extractor.weight_candidates(window=10)

# 4. Get the 10 highest weighted candidates as keyphrases.
    keyphrases = extractor.get_n_best(n=10)
    
    for word, score in keyphrases:
        print(word)
    
    print('\n\n')

گوشی گلکسی A
کانفیگ‌های مورد بحث گوشی سامسونگ
غول کره‌ای قیمت گوشی
گوشی پرچمدار جدیدش
سامسونگ گلکسی A
قیمت گوشی
عنوان بهترین گوشی
گیگابایت فضای ذخیره‌سازی عرضه
اسپیکر جدید شیائومی قیمت
باتری گلکسی A


رابط کاربری Google Contacts
رابط کاربری بخش سربرگ صفحه‌ی مخاطبین
اپلیکیشن Google Contacts چیزی
برنامه‌ی Google Contacts
Google Contacts جزو اپلیکیشن‌های
نرم‌افزار Google Contacts
اضافه شدن ویژگی‌های جدید
تکنو گوشی هوشمند رول شدنی
رابط کاربری نوار کناری
آینده‌ی نزدیک شاهد تغییرات کوچکی


جدید بهترین گوشی سامسونگ حجمی معادل
گوشی سامسونگ مدل گلکسی S
اسپیکر جدید شیائومی قیمت
صفحه نمایش سری گلکسی S
شیائومی مدل جدیدی
خرید گوشی جدیدی
به‌روزرسانی جدید
سری گلکسی S
قابلیت جدیدی
پیش‌فروش تلویزیون‌های سری QNED ال‌جی


هدست واقعیت مجازی استفاده
دستگاه‌های واقعیت افزوده استفاده
هدست‌های واقعیت ترکیبی استفاده
دستگاه واقعیت مجازی رانندگی
هدست واقعیت مجازی
تاثیر رفتاری واقعیت مجازی
واقعیت ترکیبی ویژن پرو
هدست واقعیت ترکیبی ممکن
گرافیک واقعیت مجازی
روانشناسی واقعیت مجازی


### PositionRank Algorithm

In [9]:
from perke.unsupervised.graph_based import PositionRank

# refine all
for item in data[:4]:
    text = item['content']
    #text = " ".join([w for w in text.split() if w not in stop_words])

    # Define the set of valid part of speech tags to occur in the model.
    valid_pos_tags = {'NOUN', 'NOUN,EZ', 'ADJ', 'ADJ,EZ'}

# Define the grammar for selecting the keyphrase candidates
    grammar = r"""
        NP:
            {<NOUN>}<VERB>
        NP:
            {<DET(,EZ)?|NOUN(,EZ)?|NUM(,EZ)?|ADJ(,EZ)|PRON><DET(,EZ)|NOUN(,EZ)|NUM(,EZ)|ADJ(,EZ)|PRON>*}
            <NOUN>}{<.*(,EZ)?>
    """

# 1. Create a PositionRank extractor.
    extractor = PositionRank(valid_pos_tags=valid_pos_tags)

# 2. Load the text.
    extractor.load_text(input=text, word_normalization_method=None, universal_pos_tags=False)

# 3. Select the noun phrases up to 3 words as keyphrase candidates.
    extractor.select_candidates(grammar=grammar, maximum_word_number=3)

# 4. Weight the candidates using the sum of their word's weights
#    that are computed using random walk biased with the position of
#    the words in the text. In the graph, nodes are words (nouns
#    and adjectives only) that are connected if they co-occur in a
#    window of 10 words.
    extractor.weight_candidates(window_size=10)

# 4. Get the 10 highest weighted candidates as keyphrases.
    keyphrases = extractor.get_n_best(n=10)
    
    for word, score in keyphrases:
        print(word)
    
    print('\n\n')

گوشی گلکسی
سامسونگ
گوشی پرچمدار
گوشی
این گوشی
فضای ذخیره‌سازی
گلکسی
A
عرضه
قیمت


Google
Contacts
جزو اپلیکیشن‌های
رابط کاربری
شاهد تغییرات
گوگل
شدن ویژگی‌های
منوی همبرگری
تغییرات
اسپیکر جدید


گوشی سامسونگ
سامسونگ
صفحه نمایش سری
رفع مشکلات
به‌روزرسانی امنیتی مارس
رفع
گلکسی
S
دوربین
سری


واقعیت ترکیبی ویژن
دستگاه واقعیت مجازی
هدست واقعیت
دستگاه واقعیت
تاثیر رفتاری واقعیت
هدست‌های واقعیت
دستگاه‌های واقعیت
روانشناسی واقعیت
گرافیک واقعیت
مدت واقعیت


### MultipartiteRank Algorithm

In [10]:
from perke.unsupervised.graph_based import MultipartiteRank

# refine all
for item in data[:4]:
    text = item['content']
    #text = " ".join([w for w in text.split() if w not in stop_words])

    # Define the set of valid part of speech tags to occur in the model.
    valid_pos_tags = {'NOUN', 'ADJ'}

# 1. Create a MultipartiteRank extractor.
    extractor = MultipartiteRank(valid_pos_tags=valid_pos_tags)

# 2. Load the text.
    extractor.load_text(input=text, word_normalization_method='lemmatization')

# 3. Select the longest sequences of nouns and adjectives, that do
#    not contain punctuation marks or stopwords as candidates.
    extractor.select_candidates()

# 4. Build the Multipartite graph and weight candidates using
#    random walk, alpha controls the weight adjustment mechanism,
#    see TopicRank for metric, linkage method and threshold
#    parameters.
    extractor.weight_candidates(
        threshold=0.6,
        metric='jaccard',
        linkage_method='average',
        alpha=10000,
    )

# 4. Get the 10 highest weighted candidates as keyphrases.
    keyphrases = extractor.get_n_best(n=10)
    
    for word, score in keyphrases:
        print(word)
    
    print('\n\n')

دلار قیمت
هند عرضه
پیکربندی رم
قیمت
گیگابایت رم
مدل پایه
هند کاهش
گوشی
مگاپیکسلی
آذر


منوی همبرگری حذف
لگی کار
گوگل
برنامه مجهز
مشکل
فراموش
ویجتی
رابط کاربری Google Contacts
نشان
دردسترس قرار


دوربین
خانواده منتشر
سامسونگ
گزارش سم موبایل
فروردین
اسفند
دسترس کاربران اروپایی قرار
روز آینده
رفع
منتشر


روانشناسی واقعیت مجازی
واقعیت افزوده
هدست ویژن پرو
اپل
واقعیت ترکیبی ویژن پرو
ابزارهای باورنکردنی
انزوای اجتماعی
دستگاه مجهز
جرمی بیلنسون
کاربران


As you can see, the results of them even if had some acceptable keyphrases, but had a considerable number of **False Positives** which in the context of KeyPhrase extraction is so important for us. We don't want to have any False Positives or wrong sequences of words which are mistakenly recognized as a keyphrase by the code.

I also, have tried some combinations of these algorithms to get better results but didn't work well; and also tried one or two other libraries and unfortunately, they also hadn't acceptable results.

But after looking at the previous ones, I noticed that the TextRank and PositionRank algorithms work better for my case. So, I decided to use the TextRank alongside the grammar rules of PositionRank and use Sentence Embeddings for them to get the similarities based on cosine similarity to get more accurate results. Here is my best approach which just uses hazm library:


## <font color='green'>Best Approach : TextRank</font>
In this Notebook, we intend to extract the keyphrases of the sample text using the help of hazm library and some other NLP libraries. In all keyword extraction algorithms, the raw input text must first be normalized, tokenized, and labeled, which can is done before, but for the certainty, we do it again easily using the hazm library.

There are various algorithms for keyword extraction. In this Notebook, we have used the TextRank method. In the TextRank method, the input text is first broken down into sentences and words, and then a semantic graph is created. In this graph, each word or sentence plays the role of a node, and the edges between the nodes represent semantic relationships between them. The importance of each node is determined by the degree of its association with neighboring words and sentences. 

Let's examine further details through the code.

In [40]:
text = data[3]['content']

We don't normalize the input text using the hazm normalizer. Because, it corrupts the text and for example converts the English numbers which are for the English phrases to Persian numbers while they shouldn't be converted. We just break the text down into sentences using the tokenizer, and finally into words.

In [41]:
from hazm import *

tokenize_text = [word_tokenize(txt) for txt in sent_tokenize(text)]
tokenize_text

[['اپل',
  'سرانجام',
  'چندی',
  'پیش\u200cهدست',
  'واقعیت',
  'ترکیبی',
  'ویژن',
  'پرو',
  'را',
  'که',
  'از',
  'مدت\u200cها',
  'وعده',
  'داده\u200cبود',
  '،',
  'عرضه',
  'کرد',
  '.'],
 ['این',
  'شرکت',
  'دستگاه',
  'جدیدش',
  'را',
  'که',
  'قیمت',
  'اولیه\u200cی',
  'آن',
  '۳۵۰۰',
  'دلار',
  'است',
  '،',
  'به',
  'عنوان',
  '«',
  'کامپیوتر',
  'فضایی',
  '»',
  'و',
  'جایگزینی',
  'برای',
  'لپ',
  'تاپ\u200cها',
  'یا',
  'کامپیوترهای',
  'رومیزی',
  'استاندارد',
  'توصیف',
  'می\u200cکند',
  '.'],
 ['تبلیغات',
  'اپل',
  'افرادی',
  'را',
  'نشان',
  'می\u200cدهد',
  'که',
  'از',
  'هدست',
  'ویژن',
  'پرو',
  'برای',
  'ارسال',
  'ایمیل',
  'و',
  'سایر',
  'کارهای',
  'دو',
  'بعدی',
  'معمولی',
  'استفاده',
  'می\u200cکنند',
  'و',
  'این',
  'شرکت',
  'در',
  'بیانیه',
  'مطبوعاتی',
  'ژوئن',
  '۲۰۲۳',
  'اعلام',
  'کرد',
  'ویژن',
  'پرو',
  'برای',
  'استفاده',
  'تمام',
  'روز',
  'طراحی',
  'شده\u200cاست',
  '.'],
 ['کاربران',
  'مشتاق',
  'اولیه',
 

After loading the POS model of the hazm library which I have downloaded and put it next to the notebook, I tag each of the words using the POSTagger module.

<font color='red'>Note that because of using POS Tagger, we no longer need to remove any stop words, lemmatize the words or perform stemming on them. Cause using the POS tags, we already know the verbs or preposition, adverbs, etc. Also, we know the semantic relation of words in a sentence by that. So, in our specific task, there is no need to lemmatize the words or... .</font>

In [42]:
model_path = 'pos_tagger.model'
tagger = POSTagger(model = model_path, universal_tag=False)
token_tag_list = tagger.tag_sents(tokenize_text)
token_tag_list

[[('اپل', 'NOUN'),
  ('سرانجام', 'ADV'),
  ('چندی', 'NOUN'),
  ('پیش\u200cهدست', 'ADV'),
  ('واقعیت', 'NOUN,EZ'),
  ('ترکیبی', 'ADJ,EZ'),
  ('ویژن', 'NOUN,EZ'),
  ('پرو', 'NOUN'),
  ('را', 'ADP'),
  ('که', 'SCONJ'),
  ('از', 'ADP'),
  ('مدت\u200cها', 'NOUN'),
  ('وعده', 'NOUN'),
  ('داده\u200cبود', 'VERB'),
  ('،', 'PUNCT'),
  ('عرضه', 'NOUN'),
  ('کرد', 'VERB'),
  ('.', 'PUNCT')],
 [('این', 'DET'),
  ('شرکت', 'NOUN'),
  ('دستگاه', 'NOUN,EZ'),
  ('جدیدش', 'ADJ'),
  ('را', 'ADP'),
  ('که', 'SCONJ'),
  ('قیمت', 'NOUN,EZ'),
  ('اولیه\u200cی', 'ADJ,EZ'),
  ('آن', 'PRON'),
  ('۳۵۰۰', 'NUM'),
  ('دلار', 'NOUN'),
  ('است', 'VERB'),
  ('،', 'PUNCT'),
  ('به', 'ADP'),
  ('عنوان', 'NOUN,EZ'),
  ('«', 'PUNCT'),
  ('کامپیوتر', 'NOUN,EZ'),
  ('فضایی', 'ADJ'),
  ('»', 'PUNCT'),
  ('و', 'CCONJ'),
  ('جایگزینی', 'NOUN'),
  ('برای', 'ADP,EZ'),
  ('لپ', 'NOUN'),
  ('تاپ\u200cها', 'NOUN'),
  ('یا', 'CCONJ'),
  ('کامپیوترهای', 'NOUN,EZ'),
  ('رومیزی', 'NOUN,EZ'),
  ('استاندارد', 'ADJ'),
  ('توصیف', 'NOUN'

Using some grammar rules, which is derived after much time spent on POS tags of keyphrases and looking them carefully and extracting all the best rules with the **lowest False Positives** possible, we extract the candidates of being keyphrase.    

In [43]:
import numpy as np

grammars = [
"""
NP:
        {<NUM><NOUN>}    # Number + Noun

""",

"""
NP:
        {<NOUN><NUM>}    # Noun + Number

""",
    
"""
NP:
        {<NOUN,EZ>*<ADJ,EZ>*<NOUN>}    # Dependent Noun(s)(optional) + Dependent Adjective(s)(optional) + Noun

""",
    
"""
NP:
        {<ADJ,EZ>*<NOUN,EZ>*<NOUN>}    # Dependent Adjective(s)(optional) + Dependent Noun(s)(optional) + Noun

""",
    
"""
NP:
        {<NOUN,EZ>*<NOUN>?<ADJ>?}    # Dependent Noun(s)(optional) + Noun(optional) + Adjective(optional)

""",
    
"""
NP:
        {<NOUN,EZ>*<NOUN>?<ADJ,EZ>*}    # Dependent Noun(s)(optional) + Noun(optional) + Dependent Adjective(s)(optional)

"""
]

## you can also add your own grammar to be extracted from the text...

def extract_candidates(tagged, grammar):
    keyphrase_candidate = set()
    np_parser = nltk.RegexpParser(grammar)
    trees = np_parser.parse_sents(tagged)
    for tree in trees:
        for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):  # For each nounphrase
            # Concatenate the token with a space
            keyphrase_candidate.add(' '.join(word for word, tag in subtree.leaves()))
    keyphrase_candidate = {kp for kp in keyphrase_candidate if len(kp.split()) <= 5}
    keyphrase_candidate = list(keyphrase_candidate)
    return keyphrase_candidate

all_candidates = set()
for grammar in grammars:
    all_candidates.update(extract_candidates(token_tag_list, grammar))


all_candidates = np.array(list(all_candidates))


print(np.array(list(all_candidates)))

['استفاده\u200cی' 'جهان\u200cهای دیجیتالی' 'مدیر مؤسس آزمایشگاه تعامل'
 'مجموعه\u200cای' 'کامپیوتر فضایی' 'اسکنر' 'واقعیت افزوده'
 'رسانه\u200cهای' 'سریع\u200cتر' 'کردن اعداد' 'قیمت' 'باتری' 'چشم انسان'
 'اثراتی' 'ایجاد تجارب فراگیر' 'کامل' 'تصویری' 'بیماری شبیه'
 'حضور شخصیت انسانی' 'بهبود' 'تجربه' 'ممکن' 'کاربران مشتاق'
 'طبق مطالعه\u200cای' 'مقاله سال' 'طول مطالعه' 'معنا' 'انواع جدیدی' 'چالش'
 'طور کلی' 'ساعت' 'مانیتورهای فیزیکی' 'طور منظم' 'محیط' 'انجام' 'مارک'
 'هیجان انگیزی' 'کلیدها' 'بیلنسون' 'نویسنده\u200cی'
 'نویسندگان مقاله\u200cای' 'انحرافات بصری' 'ادراک بصری' 'صفحه نمایش کوچک'
 'کامپیوترهای رومیزی' 'دستگاه' 'کردن' 'بیرون' 'هدست' '۲۰۰۹ نشان'
 'دانشگاه هامبورگ آلمان' 'موجب پرت' 'بر\u200cکیفیت زندگی' 'نرم\u200cافزار'
 'هدست\u200cهایی' 'اجسام نزدیک' 'ابزارهای باورنکردنی' 'هدست دوچرخه'
 'شرایطی' 'مدت\u200cها' 'ایجاد تجارب' 'جرمی' 'نگرانی\u200cهای ایمنی'
 'آرایه دوربین' 'ذهن' 'طور' 'پیشرفت\u200cهای تکنولوژیکی' 'پژوهشگران'
 'دانشیار رسانه' 'فرهنگی' 'منتقل' 'حرکت چشم\u200cهای' 'کار

We load the sent2vec model of hazm for Sentence Embeddings.

In [10]:
sent2vec_model_path = 'sent2vec/sent2vec-naab.model'
sent2vec_model = SentEmbedding(sent2vec_model_path)

Using the model that was loaded in the previous step, we convert each of the candidates into a corresponding vector, and then once with the combination of all candidates of a vector, we determine it as the representative vector of the text.

In [44]:
all_candidates_vectors = [sent2vec_model[candidate] for candidate in all_candidates]
all_candidates_vectors[0:2]

[array([-2.66034249e-02, -1.43336440e-02, -5.12810331e-03,  2.10486110e-02,
        -1.17218597e-02,  1.09310756e-02,  1.71195008e-02,  9.27506387e-03,
        -5.74194528e-02,  4.22935374e-03,  4.49481681e-02, -1.97610725e-02,
        -6.79992214e-02, -1.22407014e-02, -1.48204202e-02,  1.11608468e-02,
        -1.49800219e-02, -1.07619315e-02,  1.01512494e-02,  5.04312525e-03,
        -2.42020935e-03,  2.56322194e-02,  1.25513356e-02, -2.65026968e-02,
        -3.48951519e-02,  3.09036449e-02, -2.22961642e-02,  9.61451884e-03,
         9.44961607e-03, -4.02526557e-03,  7.14873150e-03, -3.36340442e-02,
         9.39282589e-03,  6.67267814e-02,  1.43507468e-02,  5.85419452e-03,
        -4.19504568e-03, -1.12117575e-02, -1.80705506e-02, -5.35457246e-02,
        -4.81239147e-03,  2.51433272e-02, -6.63178580e-05, -1.27397319e-02,
        -3.73444706e-02, -2.83521805e-02,  5.12858341e-03,  9.42430180e-03,
         1.86034217e-02,  4.28288756e-03, -7.61001697e-03,  9.47443070e-04,
         3.4

In [45]:
candidates_concatinate = ' '.join(all_candidates)
whole_text_vector = sent2vec_model[candidates_concatinate]
whole_text_vector

array([-0.27463838, -0.78197676, -1.8013406 ,  3.3305144 , -0.9075974 ,
        0.55706936,  0.8673144 , -2.8857272 , -0.6256993 ,  0.5989665 ,
        0.24396034,  0.6014742 , -0.05235213,  0.72503   , -0.08392547,
        0.4153229 , -1.0573945 , -2.722391  ,  0.06002562, -0.10511485,
        0.97371614,  1.5629607 ,  0.6982924 ,  1.4744323 , -0.9803955 ,
        0.09432636, -0.9269793 ,  0.46194124,  1.2084222 ,  2.5076308 ,
        1.1082652 ,  1.4862756 , -2.629123  , -0.6001403 ,  1.1460842 ,
        0.3583753 ,  0.86848843,  0.24293454, -1.2398478 , -0.5182925 ,
        0.9680895 ,  0.24378446, -1.5970086 , -0.39801005, -1.0900481 ,
       -0.6923939 ,  0.37151894, -1.6021073 , -1.8680102 ,  1.3056967 ,
       -1.9318887 , -0.08233713,  1.0676106 , -0.3591833 , -0.23392928,
        0.73140734,  0.1546846 , -1.2388828 ,  1.9524629 , -1.2678112 ,
        1.1427567 , -0.03012239, -0.86639434,  0.4253501 ,  1.6676373 ,
        1.7130713 ,  0.47867092, -1.8619009 ,  0.2557363 , -1.03

We calculate the cosine similarity between each of the candidates and the representative vector of the text.

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

candidates_sim_whole = cosine_similarity(all_candidates_vectors, whole_text_vector.reshape(1,-1))
candidates_sim_whole.reshape(1,-1)

array([[ 3.72158661e-02,  1.32864803e-01,  3.10373664e-01,
         1.18473910e-01,  2.07432568e-01,  2.06849650e-01,
         2.69758385e-02,  1.32909507e-01,  6.57750890e-02,
        -2.88618412e-02,  4.82803062e-02,  1.49751991e-01,
         2.39569098e-02,  9.56196487e-02,  2.08916426e-01,
        -7.27769136e-02,  6.55865818e-02,  9.30532515e-02,
         1.13939181e-01,  2.10946947e-01,  1.68025434e-01,
         1.13341451e-01,  2.13441759e-01,  3.10560949e-02,
         8.93339962e-02,  2.79483825e-01,  5.22811562e-02,
         1.02849707e-01,  1.92179997e-02,  6.03594817e-03,
         9.26336795e-02,  2.17913747e-01, -2.06719022e-02,
         1.86816484e-01,  5.77109084e-02,  1.42777264e-01,
        -8.42807628e-03,  7.81732574e-02,  4.26047444e-02,
         3.45986485e-02,  4.95478921e-02,  2.04396442e-01,
        -3.03056240e-02,  3.75658944e-02,  3.83212604e-03,
         1.30256474e-01, -9.25456509e-02,  6.79739863e-02,
         2.34483674e-01, -4.69078198e-02,  1.56937223e-0

We create a matrix where each element represents the cosine similarity between candidate i and candidate j, indexed by i and j, respectively.

In [47]:
candidate_sim_candidate = cosine_similarity(all_candidates_vectors)
candidate_sim_candidate

array([[ 1.0000002e+00,  4.5057392e-01, -6.6445403e-02, ...,
        -9.0413459e-02,  2.5758386e-01,  2.1289523e-01],
       [ 4.5057392e-01,  9.9999994e-01, -9.0390118e-03, ...,
        -2.4495140e-01,  3.7766320e-01,  2.6525474e-01],
       [-6.6445403e-02, -9.0390118e-03,  9.9999976e-01, ...,
         5.1499605e-02, -7.0230454e-02, -1.4557576e-02],
       ...,
       [-9.0413459e-02, -2.4495140e-01,  5.1499605e-02, ...,
         9.9999988e-01,  9.5450686e-04, -2.2464186e-02],
       [ 2.5758386e-01,  3.7766320e-01, -7.0230454e-02, ...,
         9.5450686e-04,  1.0000000e+00,  2.5840259e-01],
       [ 2.1289523e-01,  2.6525474e-01, -1.4557576e-02, ...,
        -2.2464186e-02,  2.5840259e-01,  9.9999958e-01]], dtype=float32)

We normalize the two values above for use in subsequent steps.

In [48]:
candidates_sim_whole_norm = candidates_sim_whole / np.max(candidates_sim_whole)
candidates_sim_whole_norm = 0.5 + (candidates_sim_whole_norm - np.average(candidates_sim_whole_norm)) / np.std(candidates_sim_whole_norm)
candidates_sim_whole_norm

array([[-6.8493783e-02],
       [ 9.4411063e-01],
       [ 2.8233395e+00],
       [ 7.9175884e-01],
       [ 1.7335354e+00],
       [ 1.7273642e+00],
       [-1.7690170e-01],
       [ 9.4458389e-01],
       [ 2.3385346e-01],
       [-7.6803720e-01],
       [ 4.8641860e-02],
       [ 1.1228898e+00],
       [-2.0886207e-01],
       [ 5.4980820e-01],
       [ 1.7492445e+00],
       [-1.2329519e+00],
       [ 2.3185778e-01],
       [ 5.2263856e-01],
       [ 7.4375117e-01],
       [ 1.7707410e+00],
       [ 1.3163449e+00],
       [ 7.3742324e-01],
       [ 1.7971529e+00],
       [-1.3370532e-01],
       [ 4.8326403e-01],
       [ 2.4963188e+00],
       [ 9.0997577e-02],
       [ 6.2635052e-01],
       [-2.5903141e-01],
       [-3.9858556e-01],
       [ 5.1819670e-01],
       [ 1.8444963e+00],
       [-6.8133283e-01],
       [ 1.5152797e+00],
       [ 1.4848059e-01],
       [ 1.0490506e+00],
       [-5.5171156e-01],
       [ 3.6510885e-01],
       [-1.1443496e-02],
       [-9.6201420e-02],


In [49]:
np.fill_diagonal(candidate_sim_candidate, np.NaN)
candidate_sim_candidate_norm = candidate_sim_candidate / np.nanmax(candidate_sim_candidate, axis=0)
candidate_sim_candidate_norm = 0.5 + (candidate_sim_candidate_norm - np.nanmean(candidate_sim_candidate_norm, axis=0)) / np.nanstd(candidate_sim_candidate_norm, axis=0)
candidate_sim_candidate_norm

array([[        nan,  1.1794994 , -0.04947621, ...,  0.28207707,
         0.8561585 ,  0.6184779 ],
       [ 1.4974904 ,         nan,  0.5452981 , ..., -1.4487605 ,
         1.6307623 ,  0.9596091 ],
       [-1.1795145 , -0.9594135 ,         nan, ...,  1.871515  ,
        -1.2584953 , -0.8634169 ],
       ...,
       [-1.3036155 , -2.0572853 ,  1.1725247 , ...,         nan,
        -0.7992976 , -0.91492975],
       [ 0.49823305,  0.8401927 , -0.08869231, ...,  1.3054059 ,
                nan,  0.9149662 ],
       [ 0.2668458 ,  0.31707454,  0.48812154, ...,  1.0431145 ,
         0.86144   ,         nan]], dtype=float32)

Using the EmbedRank method in a recursive algorithm, at each step, a candidate is selected as the key phrase using a formula. The candidate selected should have the highest similarity with the entire text in the first priority and the lowest similarity with the selected candidates in the second priority. The effectiveness of these two factors can be adjusted by considering various factors such as text length and content. (beta)

In [50]:
beta = 0.8
keyword_count = 10

N = min(len(all_candidates), keyword_count)

selected_candidates = []
unselected_candidates = [i for i in range(len(all_candidates))]
best_candidate = np.argmax(candidates_sim_whole_norm)
selected_candidates.append(best_candidate)
unselected_candidates.remove(best_candidate)


for i in range(N-1):
    selected_vec = np.array(selected_candidates)
    unselected_vec = np.array(unselected_candidates)

    unselected_candidate_sim_whole_norm = candidates_sim_whole_norm[unselected_vec, :]

    dist_between = candidate_sim_candidate_norm[unselected_vec][:, selected_vec]

    if dist_between.ndim == 1:
        dist_between = dist_between[:, np.newaxis]

    best_candidate = np.argmax(beta * unselected_candidate_sim_whole_norm - (1 - beta) * np.max(dist_between, axis = 1).reshape(-1,1))
    best_index = unselected_candidates[best_candidate]
    selected_candidates.append(best_index)
    unselected_candidates.remove(best_index)
for keyword in all_candidates[selected_candidates].tolist():
    print(keyword)

هدست واقعیت مجازی
سبک فناوری ویديویی مشابه Pass-Through
هدست‌های واقعیت ترکیبی
کیفیت واقعیت
دستگاه واقعیت مجازی رانندگی
مدیر مؤسس آزمایشگاه تعامل
هدست واقعیت
دستگاه واقعیت مجازی
هدست‌های واقعیت
هدست واقعیت ترکیبی


## Run for All the Data

In [52]:
from hazm import *
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

beta = 0.82
keyword_count = 10

model_path = 'pos_tagger.model'
tagger = POSTagger(model = model_path, universal_tag=False)

sent2vec_model_path = 'sent2vec/sent2vec-naab.model'
sent2vec_model = SentEmbedding(sent2vec_model_path)

grammars = [
"""
NP:
        {<NUM><NOUN>}    # Number + Noun

""",

"""
NP:
        {<NOUN><NUM>}    # Noun + Number

""",
    
"""
NP:
        {<NOUN,EZ>*<ADJ,EZ>*<NOUN>}    # Dependent Noun(s)(optional) + Dependent Adjective(s)(optional) + Noun

""",
    
"""
NP:
        {<ADJ,EZ>*<NOUN,EZ>*<NOUN>}    # Dependent Adjective(s)(optional) + Dependent Noun(s)(optional) + Noun

""",
    
"""
NP:
        {<NOUN,EZ>*<NOUN>?<ADJ>?}    # Dependent Noun(s)(optional) + Noun(optional) + Adjective(optional)

""",
    
"""
NP:
        {<NOUN,EZ>*<NOUN>?<ADJ,EZ>*}    # Dependent Noun(s)(optional) + Noun(optional) + Dependent Adjective(s)(optional)

"""
]

# we can also add some additional grammar to be extracted from the text...

def extract_candidates(tagged, grammar):
    keyphrase_candidate = set()
    np_parser = nltk.RegexpParser(grammar)
    trees = np_parser.parse_sents(tagged)
    for tree in trees:
        for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):  # For each nounphrase
            # Concatenate the token with a space
            keyphrase_candidate.add(' '.join(word for word, tag in subtree.leaves()))
    keyphrase_candidate = {kp for kp in keyphrase_candidate if len(kp.split()) <= 5}
    keyphrase_candidate = list(keyphrase_candidate)
    return keyphrase_candidate

for item in data:
    text = item['content']
    try:
        tokenize_text = [word_tokenize(txt) for txt in sent_tokenize(text)]  
        
        token_tag_list = tagger.tag_sents(tokenize_text)
        all_candidates = set()
        for grammar in grammars:
            all_candidates.update(extract_candidates(token_tag_list, grammar))
        
        
        all_candidates = np.array(list(all_candidates))
        
        all_candidates_vectors = [sent2vec_model[candidate] for candidate in all_candidates]

        candidates_concatinate = ' '.join(all_candidates)
        whole_text_vector = sent2vec_model[candidates_concatinate]

        candidates_sim_whole = cosine_similarity(all_candidates_vectors, whole_text_vector.reshape(1,-1))
        
        candidate_sim_candidate = cosine_similarity(all_candidates_vectors)
        
        candidates_sim_whole_norm = candidates_sim_whole / np.max(candidates_sim_whole)
        candidates_sim_whole_norm = 0.5 + (candidates_sim_whole_norm - np.average(candidates_sim_whole_norm)) / np.std(candidates_sim_whole_norm)
        
        np.fill_diagonal(candidate_sim_candidate, np.NaN)
        candidate_sim_candidate_norm = candidate_sim_candidate / np.nanmax(candidate_sim_candidate, axis=0)
        candidate_sim_candidate_norm = 0.5 + (candidate_sim_candidate_norm - np.nanmean(candidate_sim_candidate_norm, axis=0)) / np.nanstd(candidate_sim_candidate_norm, axis=0)
        
        N = min(len(all_candidates), keyword_count)

        selected_candidates = []
        unselected_candidates = [i for i in range(len(all_candidates))]
        best_candidate = np.argmax(candidates_sim_whole_norm)
        selected_candidates.append(best_candidate)
        unselected_candidates.remove(best_candidate)
        
        
        for i in range(N-1):
            selected_vec = np.array(selected_candidates)
            unselected_vec = np.array(unselected_candidates)
        
            unselected_candidate_sim_whole_norm = candidates_sim_whole_norm[unselected_vec, :]
        
            dist_between = candidate_sim_candidate_norm[unselected_vec][:, selected_vec]
        
            if dist_between.ndim == 1:
                dist_between = dist_between[:, np.newaxis]
        
            best_candidate = np.argmax(beta * unselected_candidate_sim_whole_norm - (1 - beta) * np.max(dist_between, axis = 1).reshape(-1,1))
            best_index = unselected_candidates[best_candidate]
            selected_candidates.append(best_index)
            unselected_candidates.remove(best_index)
            
        keywords = all_candidates[selected_candidates].tolist()
        
        item['keywords'] = keywords
        
    except:
        print('The code could not extract the desired keywords from the article')
        item['keywords'] = ['']
    
    
with open('final_processed_data.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

The code could not extract the desired keywords from the article
The code could not extract the desired keywords from the article


Now, we are going to show 5 random samples of the final result file 

In [54]:
import random

for i, item in enumerate(random.sample(data, 5)):
    print(f'{i+1})\n')
    print(f'Title of the article:\n {item["title"]}\n\n')
    print(f'Processed (normalized) content of the article:\n {item["content"]}\n\n')
    print(f'Extracted keywords of the article:\n')
    for keyword in item['keywords']:
        print(keyword)
    print('\n\n')
    

1)

Title of the article:
 گوشی ارزان نسبتاً قدیمی شیائومی به HyperOS آپدیت شد


Processed (normalized) content of the article:
  شیائومی رابط کاربری HyperOS را سال گذشته معرفی کرد تا از MIUI فاصله بگیرد. از آن زمان، HyperOS برای برخی از مدل‌های گوشی شیائومی منتشر شده‌است؛ اما همچنان اکثر دستگاه‌ها این رابط کاربری مبتنی بر اندروید را ندارند. براساس اعلام کانال تلگرام وابسته به شیائومی، رابط کاربری هایپر او اس دردسترس گوشی Redmi Note 11 Pro 4G قرار گرفته‌است. این گوشی که در سال ۲۰۲۱ با MIUI 13 و سیستم عامل اندروید ۱۱ روانه‌ی بازار شد، به پردازنده‌ی هلیو G96 مدیاتک مجهز است. بررسی‌های اولیه نشان می‌دهد که HyperOS در گوشی ردمی نوت ۱۱ پرو برخی تفاوت‌ها با همین رابط کاربری در گوشی‌های پرچم‌دار شیائومی دارد. مهم‌ترین تفاوت این است که گوشی یادشده از نسخه‌ی اندروید ۱۳ رابط کاربری HyperOS استفاده می‌کند. همچنین، از انتشار اندروید ۱۴ برای ردمی نوت ۱۱ پرو 4G خبری نخواهد بود. در حال حاضر، به روزرسانی HyperOS صرفاً دردسترس کاربران Mi Pilot قرار دارد و به صورت عمومی منتشر نشده‌است. به روزرسانی مذکور

As it can be seen easily, the keywords are correct and highly related to the title and descriptions of the article.